In [7]:
from yahoo import ylib
from pathlib import Path
from datetime import date
import pandas as pd

In [12]:
import requests
from bs4 import BeautifulSoup
import urllib
import time

PAGE_START = 1
PAGE_END = 6
COLUMNS = ['rank','ticker','market','name','date_time','stock_price','fiscal_month','DPS','dividend','掲示板']

# P1 - P6(300社分) HTML取得
def get_ranking_html(kd:int, out_dir:str):
    for page in list(range(PAGE_START,PAGE_END)):
        load_url = f"https://info.finance.yahoo.co.jp/ranking/?kd={kd}&tm=d&vl=a&mk=1&p={page}"
        html = requests.get(load_url)
        filename = f"{page}.html"
        out_path = out_dir.joinpath(filename)
        with open(out_path, mode="wb") as f:
            f.write(html.content)
        time.sleep

def get_stocks(out_dir:str, page):
    filename = f"{page}.html"
    file_path = out_dir.joinpath(filename)
    with open(file_path, mode="r") as f:
        html = f.read()
        soup = BeautifulSoup(html, "html.parser")
        rankingTable = soup.find(class_="rankingTable")
        stocks = []
        for tr in rankingTable.find_all("tr", class_="rankingTabledata"):
            stock = []
            for data in tr.find_all("td"):
                stock.append(data.text)
            stocks.append(stock)
    return stocks

In [63]:
# generate high_diviends html from yahoo
# 8 : high_diviends
kd_idx = 8
kd_name = "high_diviends"

day = date.today().strftime("%Y%m%d")
out_dir = Path(f"html/{kd_name}/{day}/")
out_dir.mkdir(exist_ok=True)

# get_ranking_html(kd_idx, out_dir)
df_stock = pd.DataFrame()

# for page in list(range(PAGE_START, PAGE_END)):
df_concat = pd.DataFrame()
for page in list(range(PAGE_START, PAGE_END)):
    df_stock   = get_stocks(out_dir, 1)
    df_stock   = pd.DataFrame(df_stock, columns=COLUMNS)
    df_stock   = df_stock.drop("掲示板", axis=1)
    df_stock   = df_stock.drop("date_time", axis=1)
    df_stock["date"] = day
    df_concat = pd.concat([df_concat,df_stock], ignore_index=True)
df_concat

In [13]:
# Dynamo DB への保存

In [ ]:
# Key フォーマットの調整
df_concat["rank"] = df_concat["rank"].astype(int)

In [88]:
import boto3
from boto3.dynamodb.conditions import Key # Boto 3 SDK によって Attr が自動的に作成される

def store_stocks(stocks:pd.DataFrame, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")
    table = dynamodb.Table('HighDividends')
    for index, row in stocks.iterrows():
        table.put_item(Item=row.to_dict())

store_stocks(df_concat)